In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset
import pandas as pd

dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Task_C/task_C_train.csv', split="train")
dataset = DatasetDict({
    'train': dataset
})

tweets = load_dataset('csv', data_files='/content/test_task_C_index_2_tweet.csv', split="train")
labels = load_dataset('csv', data_files='/content/test_task_C_index_2_label.csv', split="train")

tweets_df = pd.DataFrame(tweets)
labels_df = pd.DataFrame(labels)

merged_df = pd.merge(tweets_df, labels_df, on='index')

valid_data = Dataset.from_pandas(merged_df)
valid_data = DatasetDict({
    'valid': valid_data
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = dataset["train"]
train_data

In [ ]:
valid_data = valid_data['valid']
valid_data

In [ ]:
alpaca_prompt = """Below is an instruction that describes the task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
The task is to identify the targets of hate speech in a given hateful text. If the hate speech is targeted towards an individual, return 0. If it is targeted towards an organization, return 1. Else, if it is targeted towards a community, return 2.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
  inputs = examples["tweet"]
  outputs = examples["label"]
  texts = []
  for input, output in zip(inputs, outputs):
      # Must add EOS_TOKEN, otherwise your generation will go on forever!
      text = alpaca_prompt.format(input, output) + EOS_TOKEN
      texts.append(text)

  return { "text" : texts, }

In [ ]:
formatted_train_data = train_data.map(formatting_prompts_func, batched=True)
formatted_valid_data = valid_data.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/475 [00:00<?, ? examples/s]

In [ ]:
formatted_train_data

In [ ]:
formatted_valid_data

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_data,
    eval_dataset = formatted_valid_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 1,
        num_train_epochs = 4,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/MyDrive/Task_C/outputs_latest_qwen",
        save_steps=500,       # Save the model checkpoint every logging step
        save_total_limit=10,
        # eval_steps=200,               # Evaluate and save checkpoints every 10 steps
        # do_eval=True                 # Perform evaluation at the end of training
    ),
)

In [ ]:
trainer_stats = trainer.train()
model.save_pretrained("/content/drive/MyDrive/Task_B/Nemo_task_C_4_epochs") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Task_B/Nemo_task_C_4_epochs")

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        formatted_valid_data[3]["tweet"], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs)
res = tokenizer.decode(outputs[0])
res
pred = res.split(tokenizer.eos_token)[0][-1]
pred

'Below is an instruction that describes the task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe task is to identify the targets of hate speech in a given hateful text. If the hate speech is targeted towards an individual, return 0. If it is targeted towards an organization, return 1. Else, if it is targeted towards a community, return 2.\n\n### Input:\n@prakashgiriji नेपाली जनता श्रीलंकाको जनता बन्न समय लाग्छ। नेपाली जनताले दुख भोग्न पुगेको छैन।\n\n### Response:\n2<|im_end|>'

In [ ]:
import json
FastLanguageModel.for_inference(model)
with open('/content/drive/MyDrive/Task_C/Paper_Test/Qwen25_taskC_7th_Nov.json', 'w') as f:
  box = {}
  for i in range(0, len(formatted_valid_data)):
    obj = formatted_valid_data[i]
    print(f"Data {i}")
    inputs = tokenizer([alpaca_prompt.format(
          obj["tweet"], # input
          "", # output - leave this blank for generation!
    )], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs)
    res = tokenizer.decode(outputs[0])
    pred = res.split(tokenizer.eos_token)[0][-1]
    print(f"Pred is {pred}")
    box["tweet"] = obj["tweet"]
    box['index'] = obj['index']
    box['prediction'] = int(pred)
    box['label'] = obj['label']

    f.write(json.dumps(box))
    f.write("\n")

Data 0
Pred is 0
Data 1
Pred is 1
Data 2
Pred is 0
Data 3
Pred is 2
Data 4
Pred is 1
Data 5
Pred is 1
Data 6
Pred is 0
Data 7
Pred is 0
Data 8
Pred is 0
Data 9
Pred is 1
Data 10
Pred is 1
Data 11
Pred is 1
Data 12
Pred is 0
Data 13
Pred is 1
Data 14
Pred is 1
Data 15
Pred is 1
Data 16
Pred is 1
Data 17
Pred is 0
Data 18
Pred is 1
Data 19
Pred is 0
Data 20
Pred is 1
Data 21
Pred is 0
Data 22
Pred is 1
Data 23
Pred is 2
Data 24
Pred is 1
Data 25
Pred is 0
Data 26
Pred is 0
Data 27
Pred is 2
Data 28
Pred is 0
Data 29
Pred is 1
Data 30
Pred is 1
Data 31
Pred is 0
Data 32
Pred is 0
Data 33
Pred is 1
Data 34
Pred is 0
Data 35
Pred is 2
Data 36
Pred is 0
Data 37
Pred is 0
Data 38
Pred is 1
Data 39
Pred is 1
Data 40
Pred is 1
Data 41
Pred is 1
Data 42
Pred is 0
Data 43
Pred is 0
Data 44
Pred is 1
Data 45
Pred is 2
Data 46
Pred is 0
Data 47
Pred is 0
Data 48
Pred is 2
Data 49
Pred is 0
Data 50
Pred is 1
Data 51
Pred is 1
Data 52
Pred is 0
Data 53
Pred is 1
Data 54
Pred is 1
Data 55
Pred is 2
Da